<h2>Dataset : https://drive.google.com/file/d/1bczU3_-1t2Bb_5JJ_XCG6inSoOT7f0En/view?usp=share_link</h2>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
column_names=['userId','productId','rating','timestamp']
df=pd.read_csv('ratings_Electronics .csv',names=column_names)

In [41]:
df.shape

(7824482, 4)

In [42]:
df.head()

,userId,productId,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [43]:
df.isna().sum()

userId       0
productId    0
rating       0
timestamp    0
dtype: int64

In [44]:
review_counts = df['userId'].value_counts()
users = review_counts[review_counts >= 50].index.tolist()
df = df[df['userId'].isin(users)]

In [25]:
df = df[df['rating'] > 3]

In [26]:
df.shape

(103329, 4)

In [27]:
df.drop(columns=['timestamp'], inplace = True)

In [28]:
df.shape

(103329, 3)

In [29]:
from surprise import SVD, Reader, Dataset 
from surprise.model_selection import train_test_split

In [30]:
sim_options = {'name': 'pearson', 'user_based': False}
svd = SVD()
print(svd)
svd.sim_options = sim_options
reader = Reader()

In [31]:
data = Dataset.load_from_df(df[['userId', 'productId', 'rating']], reader)

In [32]:
trainset, testset = train_test_split(data, test_size=.25)

In [37]:
svd.fit(trainset)

In [38]:
svd.qi

array([[-0.05228554,  0.06696161,  0.16537312, ...,  0.11271912,
        -0.03995544, -0.03584349],
       [ 0.08401008,  0.13647718, -0.06665783, ..., -0.04109713,
         0.01724435,  0.03130982],
       [-0.16638704, -0.00827859, -0.07996475, ..., -0.16622007,
        -0.17121637, -0.06452454],
       ...,
       [ 0.09867366, -0.12615351, -0.02068442, ..., -0.13116117,
        -0.04057565,  0.04052813],
       [-0.03814591, -0.02898708, -0.21589361, ..., -0.05329857,
         0.11178777, -0.00284372],
       [-0.0786257 ,  0.05305619, -0.16586746, ..., -0.06345987,
        -0.13881654, -0.05724047]])

In [39]:
svd.pu

array([[-0.09283042,  0.11960867, -0.10929544, ..., -0.07620519,
         0.1055965 , -0.0065822 ],
       [-0.08671224, -0.07721008,  0.02500387, ...,  0.09309774,
         0.01098084, -0.03440183],
       [-0.04504406,  0.08833575, -0.03458067, ...,  0.04807032,
        -0.01074006, -0.13711267],
       ...,
       [ 0.04875785,  0.09216028, -0.16804116, ...,  0.02942101,
         0.00185853, -0.01985734],
       [ 0.07274501,  0.10856561, -0.03802516, ...,  0.09237136,
        -0.02901327, -0.0252563 ],
       [-0.02915766, -0.06428593, -0.01381644, ...,  0.03002306,
        -0.05682466, -0.03132497]])

In [34]:
predictions = svd.test(testset)

In [35]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.4311


0.4310972404109803

In [36]:
user_id = 'A2NYK9KWFMJV4Y'
user_items = df[df['userId'] == user_id]['productId'].unique()
all_items = df['productId'].unique()
items_to_predict = set(all_items) - set(user_items)
predictions = []
for item_id in items_to_predict:
    predicted_rating = svd.predict(user_id, item_id).est
    predictions.append((item_id, predicted_rating))
predictions.sort(key=lambda x: x[1], reverse=True)
print('Top 5 recommended items for user', user_id)
for i, (item_id, predicted_rating) in enumerate(predictions[:5]):
    print(f'{i+1}. {item_id} (predicted rating: {predicted_rating:.2f})')

Top 5 recommended items for user A2NYK9KWFMJV4Y
1. B000U92DLA (predicted rating: 5.00)
2. B001GS8G2E (predicted rating: 5.00)
3. B00CRJWW2G (predicted rating: 5.00)
4. B00009V4GX (predicted rating: 5.00)
5. B003Y0CXX0 (predicted rating: 5.00)
